# U-Net MODEL
A network architecture called "U-Net". The name of this network architecture comes from it's U-like shape when shown in a diagram like this (image from [U-net entry on wikipedia](https://en.wikipedia.org/wiki/U-Net)): 

<img src="images/U-net_example_wikipedia.png" alt="U-net Image" width="600"/>

U-nets are commonly used for image segmentation. This architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network.

In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install keras.engine

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras.engine (from versions: none)
ERROR: No matching distribution found for keras.engine

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate

# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## 1. The "Depth" of U-Net
the depth of U-Net is equal to the number of down-convolutions. In the image above, the depth is 4 because there are 4 down-convolutions running down the left side including the very bottom of the U.
Here we'll use a U-Net depth of 2, meaning we'll have 2 down-convolutions in our network.

### 1.1 Input layers and its "depth"

Here we will be doing 3D image segmentation whcih has "length" in addition to "height" and "width".

The shape of imput layer is (num_channels, height, widht, length), where num_channles is like color channels in an image, height, widht and length are just the size of the inpput

here,
num_channels = 4, 
height = 160,
width = 160,
length = 160

In [3]:
# defining an input layer tensor of the shape we'll use

input_layer = Input(shape=(4,160,160,16))
input_layer

<KerasTensor shape=(None, 4, 160, 160, 16), dtype=float32, sparse=None, name=keras_tensor>

Keras shape has "None" or "?" as the very first dimension. this will bethe batch size. so the dimensions of the tensor are: (batch_size, num_channles, height, width, length)

## 2. Contracting (downward) path

we'll start by constructing  the downward path in our network (the left side). the (height, width, length) if the input gets smaller as we move down this path, and the number of channels increases

## 2. Contracting (downward) Path 
Here you'll start by constructing the downward path in your network (the left side of the U-Net).  The `(height, width, length)` of the input gets smaller as you move down this path, and the number of channels increases.

### 2.1 Depth 0

By "depth 0" here, we're referring to the depth of the first down-convolution in the U-net.

The number of filters is specified for each depth and for each layer( layer 1, 2 etc) within that depth.

The formula to use for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=0$:
$$filters_{0} = 32 \times (2^{0}) = 32$$

### 2.2 Layer 0
There are two convolutional layers for each depth

In [4]:
# defining a Conv3D tensor with 32 filters

down_depth_0_layer_0 = Conv3D(filters=32,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(input_layer)
# constructing this on input layer 
down_depth_0_layer_0

<KerasTensor shape=(None, 32, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_1>

notice that with 32 filters, the result we got abpve is a tensor with 32 channels

In [5]:
# add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<KerasTensor shape=(None, 32, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_2>

### 2.3 Depth 0, Layer 1
For layer 1 of depth 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth. 
- Notice that the '$\times~2$' at the end of this expression isn't there for layer 0.


So at depth $i=0$ for layer 1:
$$filters_{0} = 32 \times (2^{0}) \times 2 = 64$$


In [6]:
# creating a Conv3D layer with 64 filters and add relu activation

down_depth_0_layer_1 = Conv3D(filters=64,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(down_depth_0_layer_0)
#building this over down_depth_0_layer_0

#applying activation to it
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_4>

### 2.4 Max pooling
Within the U-Net architecture, there is a max pooling operation after each of the down-convolutions (not including the last down-convolution at the bottom of the U). In general, this means we'll add max pooling after each down-convolution up to (but not including) the `depth - 1` down-convolution (since you started counting at 0). 

For this lab exercise:
- The overall depth of the U-Net we're constructing is 2
- So the bottom of our U is at a depth index of: $2-1 = 1$.
- So far we've only defined the $depth=0$ down-convolutions, so the next thing to do is add max pooling

In [7]:
# defining a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<KerasTensor shape=(None, 64, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_5>

#### 2.4.1 Depth 1, Layer 0

At depth 1, layer 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=1$:
$$filters_{1} = 32 \times (2^{1}) = 64$$



In [8]:
# Add a Conv3D layer to the network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
# building this on max pool layer
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<KerasTensor shape=(None, 64, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_7>

#### 2.4.2 Depth 1,  Layer 1

For layer 1 of depth 1 the formula we'll use for number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth. 
- Notice that the '$\times 2$' at the end of this expression isn't there for layer 0.

So at depth $i=1$:
$$filters_{0} = 32 \times (2^{1}) \times 2 = 128$$


In [9]:
# Add another Conv3D with 128 filters to the network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<KerasTensor shape=(None, 128, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_9>

#### No max pooling at depth 1 (the bottom of the U)

When we get to the "bottom" of the U-net, we don't need to apply max pooling after the convolutions.

## 3. Expanding  (upward) Path

Now we'll work on the expanding path of the U-Net, (going up on the right side, when viewing the diagram).  The image's (height, width, length) all get larger in the expanding path.

### 3.1 Depth 0, Up Sampling Layer 0

we'll use a pool size of (2,2,2) for upsampling.
- This is the default value for [tf.keras.layers.UpSampling3D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling3D)
- As input to the upsampling at depth 1, we'll use the last layer of the downsampling.  In this case, it's the depth 1 layer 1.


In [10]:
# adding anupsampling operation to the newtork
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<KerasTensor shape=(None, 128, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_10>

### 3.2 Concatenate Upsampled Depth 0 with Downsampled Depth 0

Now we'll apply a concatenation operation using the layers that are both at the same depth of 0.
- up_depth_0_layer_0: shape is (?, 128, 160, 160, 16)
- depth_0_layer_1: shape is (?, 64, 160, 160, 16)

- Double check that both of these layers have the same height, width and length.
- If they're the same, then they can be concatenated along axis 1 (the channel axis).
- The (height, width, length) is (160, 160, 16) for both.


In [11]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print()
print(down_depth_0_layer_1)

<KerasTensor shape=(None, 128, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_10>

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_4>


In [18]:
# add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,down_depth_0_layer_1],axis=1)
up_depth_1_concat

<KerasTensor shape=(None, 192, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_17>

Notice that the upsampling layer had 128 channels, and the down-convolution layer had 64 channels so that when concatenated, the result has 128 + 64 = 192 channels.

### 3.3 Up-convolution Layer 1

The number of filters for this layer will be set to the number of channels in the down-convolution's layer 1 at the same depth of 0 (down_depth_0_layer_1).


In [13]:
down_depth_0_layer_1

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_4>

Notice the number of channels for `depth_0_layer_1` is 64

In [14]:
print(f"number of filters: {down_depth_0_layer_1.shape[1]}")

number of filters: 64


In [19]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)

up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_19>

### 3.4 Up-convolution Depth 0, Layer 2

At layer 2 of depth 0 in the up-convolution the next step will be to add another up-convolution. The number of filters we'll want to use for this next up-convolution will need to be equal to the number of filters in the down-convolution depth 0 layer 1.


In [21]:
print(down_depth_0_layer_1)
print(f"number of filters: {down_depth_0_layer_1.shape[1]}")

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_4>
number of filters: 64


the number of channels / filters in `down_depth_0_layer_1` is 64.

In [22]:
# adding a conv3D up conv with 64 filters to the newtork

up_depth_1_layer_2 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_21>

## 4. Final Convolution

For the final convolution, we will set the number of filters to be equal to the number of classes in your input data.

we will be using data with 3 classes, namely:

- 1: edema
- 2: non-enhancing tumor 
- 3: enhancing tumor


In [23]:
# Add a final Conv3D with 3 filters to your network.
final_conv = Conv3D(filters=3, #3 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)
final_conv

<KerasTensor shape=(None, 3, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_22>

### 4.1 Activation for Final Convolution

In [24]:
# Add a sigmoid activation to your final convolution.
final_activation = Activation('sigmoid')(final_conv)
final_activation

<KerasTensor shape=(None, 3, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_23>

### 4.2 Create and Compile the Model

we will be setting the loss and metrics to options that are pre-built in Keras.

In [26]:
# defining and compiling the model
model = Model(inputs=input_layer, outputs = final_activation)
model.compile(optimizer=Adam(0.00001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
              )

In [27]:
# Print out a summary of the model you created
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 160,    │          0 │ -                 │
│ (InputLayer)        │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 32, 160,   │      3,488 │ input_layer[0][0] │
│                     │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 160,   │          0 │ conv3d[0][0]      │
│ (Activation)        │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 64, 160,   │     55,360 │ activation[0][0]  │
│                     │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64, 160,   │          0 │ conv3d_1[0][0]    │
│ (Activation)        │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 64, 80,    │          0 │ activation_1[0][… │
│ (MaxPooling3D)      │ 80, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 64, 80,    │    110,656 │ max_pooling3d[0]… │
│                     │ 80, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 80,    │          0 │ conv3d_2[0][0]    │
│ (Activation)        │ 80, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 128, 80,   │    221,312 │ activation_2[0][… │
│                     │ 80, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 80,   │          0 │ conv3d_3[0][0]    │
│ (Activation)        │ 80, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling3d       │ (None, 128, 160,  │          0 │ activation_3[0][… │
│ (UpSampling3D)      │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 192, 160,  │          0 │ up_sampling3d[0]… │
│ (Concatenate)       │ 160, 16)          │            │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_6 (Conv3D)   │ (None, 64, 160,   │    331,840 │ concatenate_2[0]… │
│                     │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 64, 160,   │          0 │ conv3d_6[0][0]    │
│ (Activation)        │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_7 (Conv3D)   │ (None, 64, 160,   │    110,656 │ activation_6[0][… │
│                     │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 64, 160,   │          0 │ conv3d_7[0][0]    │
│ (Activation)        │ 160, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_8 (Conv3D)   │ (None, 3, 160,    │        195 │ activation_7[0][

 Total params: 833,507 (3.18 MB)

 Trainable params: 833,507 (3.18 MB)

 Non-trainable params: 0 (0.00 B)

we've created the U-Net model acrhitecutre!